In [1]:
from nba_api.stats.static import teams
from nba_api.live.nba.endpoints import scoreboard
from utils import fetch_data, fetch_player_stats
from nba_api.stats.endpoints import teamdetails, teamyearbyyearstats, commonteamroster, teamgamelog, leaguegamefinder
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta
import re

2024-10-26 17:36:34.840 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-10-26 17:36:34.848 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


## Basics

https://github.com/swar/nba_api/blob/master/docs/examples/Basics.ipynb

## Teams

In [3]:
team_id = 1610612738 # celtics https://www.nba.com/stats/team/1610612738
# previous_season = datetime.now().year - 1
# game_log = fetch_data(teamgamelog.TeamGameLog, team_id=team_id, season=previous_season)
game_log = teamgamelog.TeamGameLog(team_id=team_id, season='2024-25').get_data_frames()[0]
game_log

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612738,0022400073,"OCT 24, 2024",BOS @ WAS,W,2,0,1.0,240,42,...,0.778,12,37,49,21,7,3,14,19,122
1,1610612738,0022400061,"OCT 22, 2024",BOS vs. NYK,W,1,0,1.0,240,48,...,0.875,11,29,40,33,6,3,3,15,132


In [4]:
g = game_log.iloc[0]
g

Team_ID        1610612738
Game_ID        0022400073
GAME_DATE    OCT 24, 2024
MATCHUP         BOS @ WAS
WL                      W
W                       2
L                       0
W_PCT                 1.0
MIN                   240
FGM                    42
FGA                    90
FG_PCT              0.467
FG3M                   17
FG3A                   45
FG3_PCT             0.378
FTM                    21
FTA                    27
FT_PCT              0.778
OREB                   12
DREB                   37
REB                    49
AST                    21
STL                     7
BLK                     3
TOV                    14
PF                     19
PTS                   122
Name: 0, dtype: object

In [5]:
matchup = g['MATCHUP'].replace(' vs. ', '-vs-').replace(' @ ', '-vs-').lower()
game_id = g['Game_ID']
print(f"https://www.nba.com/game/{matchup}-{game_id}")

https://www.nba.com/game/bos-vs-was-0022400073


In [6]:
league_game_finder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00', season_nullable='2023-24')
games = league_game_finder.get_data_frames()[0]
games[games['GAME_ID'] == '0022300563']


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
1470,22023,1610612738,BOS,Boston Celtics,0022300563,2024-01-15,BOS @ TOR,W,240,105,...,0.920,5,48,53,20,3,7,12,16,9.0
1476,22023,1610612761,TOR,Toronto Raptors,0022300563,2024-01-15,TOR vs. BOS,L,241,96,...,0.762,9,43,52,25,6,2,7,21,-9.0


## Live Data


https://github.com/swar/nba_api/blob/master/docs/examples/LiveData.ipynb

In [8]:
games = scoreboard.ScoreBoard()
# games.get_dict()
games.get_dict()['scoreboard']['games'][0]


{'gameId': '0022400087',
 'gameCode': '20241026/LACDEN',
 'gameStatus': 2,
 'gameStatusText': 'Q2 9:45',
 'period': 2,
 'gameClock': 'PT09M45.00S',
 'gameTimeUTC': '2024-10-26T21:00:00Z',
 'gameEt': '2024-10-26T17:00:00-04:00',
 'regulationPeriods': 4,
 'ifNecessary': False,
 'seriesGameNumber': '',
 'gameLabel': None,
 'gameSubLabel': None,
 'seriesText': '',
 'seriesConference': '',
 'poRoundDesc': '',
 'gameSubtype': '',
 'homeTeam': {'teamId': 1610612743,
  'teamName': 'Nuggets',
  'teamCity': 'Denver',
  'teamTricode': 'DEN',
  'wins': 0,
  'losses': 1,
  'score': 23,
  'seed': None,
  'inBonus': '0',
  'timeoutsRemaining': 6,
  'periods': [{'period': 1, 'periodType': 'REGULAR', 'score': 19},
   {'period': 2, 'periodType': 'REGULAR', 'score': 4},
   {'period': 3, 'periodType': 'REGULAR', 'score': 0},
   {'period': 4, 'periodType': 'REGULAR', 'score': 0}]},
 'awayTeam': {'teamId': 1610612746,
  'teamName': 'Clippers',
  'teamCity': 'LA',
  'teamTricode': 'LAC',
  'wins': 0,
  'loss

In [33]:
import pandas as pd
from nba_api.live.nba.endpoints import scoreboard
from datetime import datetime

def calculate_win_probability(elo_a, elo_b):
    return 1 / (1 + 10 ** ((elo_b - elo_a) / 400))

def get_today_games():
    today = datetime.now().date()
    date_str = today.strftime("%Y-%m-%d")
    
    # Load ELO ratings
    elo_df = pd.read_csv('elo-2023-24.csv')
    elo_dict = dict(zip(elo_df['Team'], elo_df['Elo']))
    
    board = scoreboard.ScoreBoard()
    games_today = board.games.get_dict()
    
    games = []
    for game in games_today:
        home_team = game['homeTeam']['teamName']
        away_team = game['awayTeam']['teamName']
        
        home_elo = elo_dict.get(home_team, 1500)  # Default ELO if not found
        away_elo = elo_dict.get(away_team, 1500)  # Default ELO if not found
        
        home_win_prob = round(calculate_win_probability(home_elo, away_elo), 2)
        away_win_prob = round(1 - home_win_prob, 2)
        
        game_info = {
            'Game ID': game['gameId'],
            'Date': date_str,
            'Home Team': home_team,
            'Away Team': away_team,
            # 'Home ELO': home_elo,
            # 'Away ELO': away_elo,
            'Home Win Probability': home_win_prob,
            'Away Win Probability': away_win_prob
        }
        games.append(game_info)
    
    return pd.DataFrame(games)

get_today_games()

,Game ID,Date,Home Team,Away Team,Home Win Probability,Away Win Probability
0,0022400087,2024-10-26,Nuggets,Clippers,0.70,0.30
1,0022400088,2024-10-26,Hornets,Heat,0.21,0.79
2,0022400089,2024-10-26,Pistons,Celtics,0.07,0.93
3,0022400090,2024-10-26,Wizards,Cavaliers,0.17,0.83
4,0022400091,2024-10-26,Bulls,Thunder,0.25,0.75
5,0022400092,2024-10-26,Grizzlies,Magic,0.22,0.78
6,0022400093,2024-10-26,Timberwolves,Raptors,0.89,0.11
7,0022400094,2024-10-26,Spurs,Rockets,0.34,0.66
8,0022400095,2024-10-26,Suns,Mavericks,0.38,0.62
9,0022400096,2024-10-26,Lakers,Kings,0.54,0.46


In [9]:
# Query nba.live.endpoints.scoreboard and  list games in localTimeZone
from datetime import datetime, timezone
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard

f = "{gameId}: {awayTeam} vs. {homeTeam} @ {gameTimeLTZ}" 

board = scoreboard.ScoreBoard()
print("ScoreBoardDate: " + board.score_board_date)
games = board.games.get_dict()
for game in games:
    gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
    print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))

ScoreBoardDate: 2024-10-14
0012400046: 76ers vs. Hawks @ 2024-10-14 19:30:00-04:00
0012400045: Grizzlies vs. Pacers @ 2024-10-14 19:00:00-04:00
0012400047: Wizards vs. Nets @ 2024-10-14 19:30:00-04:00
0012400048: Bulls vs. Bucks @ 2024-10-14 20:00:00-04:00
0012400049: Mavericks vs. Clippers @ 2024-10-14 22:30:00-04:00


In [27]:
# Getting Box Scores. 
# Get BoxScore
from nba_api.live.nba.endpoints import boxscore
box = boxscore.BoxScore('0012400045') 
# Note: home_team & away_team have the identicial data structure.
away_team = box.away_team.get_dict()['teamName']
players = box.away_team.get_dict()['players']
f = "[{name}](https://www.nba.com/stats/player/{player_id}) ({team}): {points} PTS"
for player in players:
    print(f.format(name=player['name'],player_id=player['personId'],team=away_team,points=player['statistics']['points']))

[Jake LaRavia](https://www.nba.com/stats/player/1631222) (Grizzlies): 3 PTS
[Santi Aldama](https://www.nba.com/stats/player/1630583) (Grizzlies): 3 PTS
[Brandon Clarke](https://www.nba.com/stats/player/1629634) (Grizzlies): 2 PTS
[Desmond Bane](https://www.nba.com/stats/player/1630217) (Grizzlies): 11 PTS
[Scotty Pippen Jr.](https://www.nba.com/stats/player/1630590) (Grizzlies): 3 PTS
[Luke Kennard](https://www.nba.com/stats/player/1628379) (Grizzlies): 3 PTS
[Jaylen Wells](https://www.nba.com/stats/player/1642377) (Grizzlies): 2 PTS
[Zach Edey](https://www.nba.com/stats/player/1641744) (Grizzlies): 6 PTS
[Yuki Kawamura](https://www.nba.com/stats/player/1642530) (Grizzlies): 3 PTS
[John Konchar](https://www.nba.com/stats/player/1629723) (Grizzlies): 2 PTS
[Armando Bacot](https://www.nba.com/stats/player/1630654) (Grizzlies): 0 PTS
[Jay Huff](https://www.nba.com/stats/player/1630643) (Grizzlies): 0 PTS
[GG Jackson](https://www.nba.com/stats/player/1641713) (Grizzlies): 0 PTS
[Jaren Jack

## Finding Games

https://github.com/swar/nba_api/blob/master/docs/examples/Finding%20Games.ipynb

In [7]:
from nba_api.stats.endpoints import leaguegamefinder
league_game_finder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00', season_nullable='2024-25')
games = league_game_finder.get_data_frames()[0]
games


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612743,DEN,Denver Nuggets,0022400087,2024-10-26,DEN vs. LAC,None,60,19,...,0.500,1,7,8,1,3,0,1,4,-8.0
1,22024,1610612746,LAC,LA Clippers,0022400087,2024-10-26,LAC @ DEN,None,48,27,...,1.000,1,15,16,10,1,0,3,2,8.0
2,22024,1610612755,PHI,Philadelphia 76ers,0022400078,2024-10-25,PHI @ TOR,L,238,107,...,0.755,10,18,28,19,11,5,16,32,-8.0
3,22024,1610612737,ATL,Atlanta Hawks,0022400079,2024-10-25,ATL vs. CHA,W,240,125,...,0.868,7,32,39,25,11,9,13,28,5.0
4,22024,1610612740,NOP,New Orleans Pelicans,0022400086,2024-10-25,NOP @ POR,W,238,105,...,0.677,11,33,44,25,12,8,14,22,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,12024,1610612750,MIN,Minnesota Timberwolves,0012400003,2024-10-04,MIN @ LAL,W,240,124,...,0.577,9,32,41,24,13,3,14,19,17.0
196,12024,15020,NZB,New Zealand Breakers,0012400002,2024-10-04,NZB @ UTA,L,239,87,...,0.615,13,22,35,23,11,5,16,20,-29.0
197,12024,1610612747,LAL,Los Angeles Lakers,0012400003,2024-10-04,LAL vs. MIN,L,241,107,...,0.913,4,34,38,28,9,12,20,20,-17.0
198,12024,1610612762,UTA,Utah Jazz,0012400002,2024-10-04,UTA vs. NZB,W,238,116,...,0.727,9,35,44,27,11,5,21,19,29.0


## ELO

In [ ]:
import math

def calculate_elo(rating1, rating2, k, score1, score2):
    """
    Calculate the new Elo ratings for two teams.
    
    :param rating1: Current rating of team 1
    :param rating2: Current rating of team 2
    :param k: K-factor (usually between 20 and 32)
    :param score1: Score of team 1
    :param score2: Score of team 2
    :return: Tuple of new ratings (rating1, rating2)
    """
    expected1 = 1 / (1 + math.pow(10, (rating2 - rating1) / 400))
    expected2 = 1 / (1 + math.pow(10, (rating1 - rating2) / 400))
    
    result1 = 1 if score1 > score2 else (0.5 if score1 == score2 else 0)
    result2 = 1 - result1
    
    new_rating1 = rating1 + k * (result1 - expected1)
    new_rating2 = rating2 + k * (result2 - expected2)
    
    return new_rating1, new_rating2

### Compute Elo

In [ ]:
def get_team_elo(team_id, games_df):
    """
    Calculate the current ELO rating for a team based on their game history.
    
    :param team_id: ID of the team
    :param games_df: DataFrame containing all games
    :return: Current ELO rating for the team
    """
    initial_elo = 1500
    k_factor = 20
    
    team_games = games_df[games_df['TEAM_ID'] == team_id].sort_values('GAME_DATE')
    current_elo = initial_elo
    
    for _, game in team_games.iterrows():
        opponent_abbr = game['MATCHUP'].split()[-1]
        opponent_games = games_df[(games_df['TEAM_ABBREVIATION'] == opponent_abbr) & (games_df['GAME_DATE'] < game['GAME_DATE'])]
        opponent_elo = initial_elo if opponent_games.empty else get_team_elo(opponent_games.iloc[0]['TEAM_ID'], opponent_games)
        
        team_points = game['PTS']
        opponent_points = game['PLUS_MINUS'] - team_points if game['WL'] == 'L' else team_points - game['PLUS_MINUS']
        
        new_elo, _ = calculate_elo(current_elo, opponent_elo, k_factor, team_points, opponent_points)
        current_elo = new_elo
    
    return current_elo

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd

team_id = 1610612738  # Celtics

# Fetch all games for the Celtics
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
celtics_games = gamefinder.get_data_frames()[0]

# Sort the games by date (most recent first)
celtics_games = celtics_games.sort_values('GAME_DATE', ascending=False)
celtics_games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612738,BOS,Boston Celtics,0022400073,2024-10-24,BOS @ WAS,W,240,122,...,0.778,12,37,49,21,7,3,14,19,20.0
1,22024,1610612738,BOS,Boston Celtics,0022400061,2024-10-22,BOS vs. NYK,W,240,132,...,0.875,11,29,40,33,6,3,3,15,23.0
2,12024,1610612738,BOS,Boston Celtics,0012400050,2024-10-15,BOS @ TOR,L,239,118,...,0.857,8,28,36,19,10,5,6,15,-1.0
3,12024,1610612738,BOS,Boston Celtics,0012400042,2024-10-13,BOS vs. TOR,W,240,115,...,0.826,18,32,50,29,7,5,13,21,4.0
4,12024,1610612738,BOS,Boston Celtics,0012400036,2024-10-12,BOS vs. PHI,W,240,139,...,0.714,16,44,60,38,9,7,13,14,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919,21983,1610612738,BOS,Boston Celtics,0028300052,1983-11-05,BOS @ WAS,W,240,120,...,0.778,5,28,33,36,7,7,16,26,NaN
3920,21983,1610612738,BOS,Boston Celtics,0028300043,1983-11-04,BOS vs. IND,W,240,121,...,0.841,11,34,45,28,8,5,22,26,NaN
3921,21983,1610612738,BOS,Boston Celtics,0028300033,1983-11-02,BOS vs. MIL,W,240,119,...,0.816,14,24,38,26,7,7,18,18,NaN
3922,21983,1610612738,BOS,Boston Celtics,0028300019,1983-10-29,BOS @ CLE,W,240,108,...,0.690,13,31,44,25,12,7,13,22,NaN


In [ ]:
celtics_team_id = 1610612738 # celtics
get_team_elo(celtics_team_id, celtics_games)

2406.791254811004

### Get elo from previous season

In [20]:
import pandas as pd
elo = pd.read_excel('elo-2023-24.xlsx',index_col=0)
elo

,Team,W,L,W%,Diff/G,Elo,Pre '24,30 Days,15 Days,Form
1,Celtics,80,21,0.792,10.7,1752,128,21,2,WWWLW
2,Mavericks,63,41,0.606,2.3,1686,228,36,−2,LLLWL
3,T-wolves,65,33,0.663,6.1,1666,149,−38,±0,LLLWL
4,Thunder,63,29,0.685,7.3,1664,150,±0,±0,LLWLL
5,Nuggets,64,30,0.681,4.3,1660,20,±0,±0,WWWLL
6,Warriors,46,37,0.554,2.3,1613,44,±0,±0,WWLWL
7,Pacers,55,45,0.550,2.5,1613,187,−19,±0,WLLLL
8,Suns,49,37,0.570,2.2,1599,55,±0,±0,WLLLL
9,Lakers,50,39,0.562,0.6,1585,15,±0,±0,LLLWL
10,Knicks,57,38,0.600,3.7,1584,11,±0,±0,LLWLL


In [26]:
# Assuming we already have the 'elo' DataFrame

# Replace 'T-wolves' with 'Timberwolves'
elo['Team'] = elo['Team'].replace('T-wolves', 'Timberwolves')

# Display the updated row to confirm the change
print("Updated row:")
print(elo[elo['Team'] == 'Timberwolves'])

# Save the updated DataFrame to a CSV file
elo.to_csv('elo-2023-24.csv', index=False)

Updated row:
           Team   W   L     W% Diff/G   Elo Pre '24 30 Days 15 Days   Form
3  Timberwolves  65  33  0.663    6.1  1666     149     −38      ±0  LLLWL


### details

In [23]:
from nba_api.stats.static import teams
import pandas as pd

# Get all teams
all_teams = teams.get_teams()

# Convert to DataFrame for easier manipulation
teams_df = pd.DataFrame(all_teams)

# Extract team names (second element of full_name)
teams_df['team_name'] = teams_df['full_name'].apply(lambda x: x.split()[-1])

# Create the teams list
teams = teams_df['team_name'].tolist()

# Sort the teams list alphabetically
teams.sort()

# Print the teams list
print("List of NBA teams:")
for team in teams:
    print(team)

print(f"\nTotal number of teams: {len(teams)}")

# # Optional: If you want to see the full DataFrame with the new 'team_name' column
# print("\nFull DataFrame:")
# print(teams_df[['id', 'full_name', 'abbreviation', 'team_name']])

List of NBA teams:
76ers
Blazers
Bucks
Bulls
Cavaliers
Celtics
Clippers
Grizzlies
Hawks
Heat
Hornets
Jazz
Kings
Knicks
Lakers
Magic
Mavericks
Nets
Nuggets
Pacers
Pelicans
Pistons
Raptors
Rockets
Spurs
Suns
Thunder
Timberwolves
Warriors
Wizards

Total number of teams: 30


In [24]:
# Assuming we already have the 'teams' list and 'elo' DataFrame

# Create a dictionary to store the results
team_presence = {}

# Check each team
for team in teams:
    # Check if the team name is in the 'Team' column of the elo DataFrame
    is_present = team in elo['Team'].values
    team_presence[team] = is_present

# Print the results
print("Team presence in ELO DataFrame:")
for team, present in team_presence.items():
    print(f"{team}: {'Present' if present else 'Not present'}")

# Count how many teams are present and not present
present_count = sum(team_presence.values())
not_present_count = len(teams) - present_count

print(f"\nSummary:")
print(f"Teams present: {present_count}")
print(f"Teams not present: {not_present_count}")

# If there are teams not present, let's list them
if not_present_count > 0:
    print("\nTeams not found in ELO DataFrame:")
    for team, present in team_presence.items():
        if not present:
            print(team)

Team presence in ELO DataFrame:
76ers: Present
Blazers: Present
Bucks: Present
Bulls: Present
Cavaliers: Present
Celtics: Present
Clippers: Present
Grizzlies: Present
Hawks: Present
Heat: Present
Hornets: Present
Jazz: Present
Kings: Present
Knicks: Present
Lakers: Present
Magic: Present
Mavericks: Present
Nets: Present
Nuggets: Present
Pacers: Present
Pelicans: Present
Pistons: Present
Raptors: Present
Rockets: Present
Spurs: Present
Suns: Present
Thunder: Present
Timberwolves: Not present
Warriors: Present
Wizards: Present

Summary:
Teams present: 29
Teams not present: 1

Teams not found in ELO DataFrame:
Timberwolves
